# 데이터 세트 사용

이전 랩에서는 *데이터 저장소*를 사용하여 중앙 집중식 클라우드 기반 데이터 액세스 기능을 제공했습니다. 이 랩에서는 *데이터 세트*에 대해 살펴봅니다. 데이터 세트는 실험과 학습에 특정 데이터를 더 쉽게 사용할 수 있도록 제공되는 추가 추상화 수준입니다.

## 작업 영역에 연결

가장 먼저 해야 하는 작업은 Azure ML SDK를 사용하여 작업 영역에 연결하는 것입니다.

> **참고**: 이전 연습을 완료한 후 Azure 구독으로 인증된 세션이 만료된 경우 다시 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 데이터 준비

이전 랩에서는 데이터 저장소를 만들었습니다. 항상 그런 것은 아니지만, 일반적으로 데이터 세트는 데이터 저장소의 데이터를 기준으로 생성됩니다.

이전 랩을 완료하지 않았으면 다음 코드를 실행하여 작업 영역의 기본 데이터 저장소에 로컬 CSV 파일 두 개를 업로드합니다. 이전 랩을 *완료*한 경우에는 파일을 업로드하면 같은 파일을 덮어쓰게 됩니다.

In [ ]:
ws.get_default_datastore().upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # /data에서 당뇨병 CSV 파일 업로드
                       target_path='diabetes-data/', # 데이터 저장소의 폴더 경로에 해당 파일 저장
                       overwrite=True, # 이름이 같은 기존 파일 바꾸기
                       show_progress=True)

## 테이블 형식 데이터 세트 만들기

데이터 세트는 특정 데이터 원본을 캡슐화하는 개체입니다. 데이터 저장소에 업로드한 당뇨병 데이터에서 데이터 세트를 만들고 첫 20개 레코드를 살펴보겠습니다. 여기서 데이터는 CSV 파일에 포함된 구조적 형식이므로 *테이블 형식* 데이터집합을 사용합니다.

In [ ]:
from azureml.core import Dataset

# 기본 데이터 저장소 가져오기
default_ds = ws.get_default_datastore()

# 데이터 저장소의 경로에서 테이블 형식 데이터 세트 만들기(시간이 다소 걸릴 수 있음)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# 처음 20개 행을 Pandas 데이터 프레임으로 표시
tab_data_set.take(20).to_pandas_dataframe()

위의 코드에 나와 있는 것처럼 테이블 형식 데이터 세트는 Pandas 데이터 프레임으로 쉽게 변환할 수 있습니다. 따라서 일반 Python 기술을 통해 데이터를 사용할 수 있습니다.

## 파일 데이터 세트 만들기

앞에서 만든 데이터 세트는 *테이블 형식* 데이터 세트입니다. 이 데이터 세트는 데이터 세트 정의에 포함된 구조적 파일의 모든 데이터가 들어 있는 데이터 프레임으로 읽을 수 있습니다. 테이블 형식 데이터의 경우 이러한 방식이 적합합니다. 하지만 비구조적 데이터를 사용해야 하는 기계 학습 시나리오도 있고, 직접 작성한 코드에서 파일의 데이터 읽기를 처리하려는 경우도 있을 수 있습니다. 이러한 경우에는 *파일* 데이터 세트를 사용할 수 있습니다. 파일 데이터 세트에서는 가상 탑재 지점에 파일 경로 목록이 작성되는데, 이 목록을 사용하여 파일의 데이터를 읽을 수 있습니다.

In [ ]:
# 데이터 저장소의 경로에서 파일 데이터 세트 만들기(시간이 다소 걸릴 수 있음)
file_data_set = Dataset.File.from_files(path=(default_ds, 'diabetes-data/*.csv'))

# 데이터 세트의 파일 가져오기
for file_path in file_data_set.to_path():
    print(file_path)

## 데이터 세트 등록

당뇨병 데이터를 참조하는 데이터 세트를 만든 후에는 작업 영역에서 실행되는 모든 실험에서 쉽게 액세스할 수 있도록 데이터 세트를 등록할 수 있습니다.

테이블 형식 데이터 세트는 **당뇨병 데이터 세트**로 등록하고 파일 데이터 세트는 **당뇨병 파일**로 등록하겠습니다.

In [ ]:
# 테이블 형식 데이터 세트 등록
try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                               name='diabetes dataset',
                               description='diabetes data',
                               tags = {'format':'CSV'},
                               create_new_version=True)
except Exception as ex:
    print(ex)

# 파일 데이터 세트 등록
try:
    file_data_set = file_data_set.register(workspace=ws, 
                               name='diabetes file dataset',
                               description='diabetes files',
                               tags = {'format':'CSV'},
                               create_new_version=True)
except Exception as ex:
    print(ex)

print('Datasets registered')

[Azure ML Studio](https://ml.azure.com)의 작업 영역 **데이터 세트** 페이지에서 데이터 세트를 확인하고 관리할 수 있습니다. 작업 영역 개체에서 데이터 세트 목록을 가져올 수도 있습니다.

In [ ]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

랩 2A 및 2B를 완료했다면 등록된 데이터 세트에는 비주얼 디자이너 도구를 사용하여 만든 변환이 포함됩니다. 그리고 이전 연습에서 *Studio* 인터페이스를 사용하여 만든 데이터 세트와 같은 이름으로 **당뇨병 데이터 세트**를 등록하면 해당 데이터 세트의 새 *버전*이 작성됩니다. 데이터 세트의 버전을 지정하는 기능을 사용하면 이전 정의를 사용하는 파이프라인이나 기존 실험을 그대로 유지하면서 데이터 세트를 다시 정의할 수 있습니다. 기본적으로는 명명된 데이터 세트의 최신 버전이 반환되지만 다음과 같이 버전 번호를 지정하여 데이터 세트의 특정 버전을 검색할 수 있습니다.

```python
dataset_v1 = Dataset.get_by_name(ws, 'diabetes dataset', version = 1)
```


## 테이블 형식 데이터 세트에서 모델 학습 진행

데이터 세트를 만들었으므로 이제 해당 데이터 세트에서 모델 학습을 시작할 수 있습니다. 스크립트를 실행하는 데 사용되는 추정기의 *입력*으로 데이터 세트를 스크립트에 전달할 수 있습니다.

아래의 두 코드 셀을 실행하여 다음 항목을 만듭니다.

1. **diabetes_training_from_tab_dataset** 폴더
2. 분류 모델을 학습시키는 스크립트. 이 스크립트는 전달된 테이블 형식 데이터 세트를 *입력*으로 사용합니다.

In [ ]:
import os

# 실험 파일용 폴더 만들기
experiment_folder = 'diabetes_training_from_tab_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 정규화 하이퍼 매개 변수 설정(스크립트에 인수로 전달됨)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 로드(입력 데이터 세트로 전달됨)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 로지스틱 회귀 모델 학습
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 출력 폴더에 저장된 메모 파일이 실험 레코드에 자동으로 업로드됨
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

이제 추정기를 만들어 스크립트를 실행하고 학습 데이터 세트용으로 명명된 *입력*을 정의할 수 있습니다. 스크립트는 이 입력을 읽습니다.

> **참고**: **Dataset** 클래스는 SDK와 함께 설치되는 **azureml-dataprep** 패키지에 정의되어 있습니다. 그런데 이 패키지에는 **to_pandas_dataframe()** 메서드에 사용되는 **pandas** 지원 옵션이 포함되어 있으므로, 학습 실험을 실행할 환경에 이 패키지를 포함해야 합니다.

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment
from azureml.widgets import RunDetails

# 스크립트 매개 변수 설정
script_params = {
    '--regularization': 0.1
}

# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes dataset")

# 추정기 만들기
estimator = SKLearn(source_directory=experiment_folder,
                    entry_script='diabetes_training.py',
                    script_params=script_params,
                    compute_target = 'local',
                    inputs=[diabetes_ds.as_named_input('diabetes')], # Dataset 개체를 입력으로 전달...
                    pip_packages=['azureml-dataprep[pandas]'] # ...dataprep 패키지 필요
                   )

# 실험 만들기
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# 실험 실행
run = experiment.submit(config=estimator)
# 실행 중에 실행 세부 정보 표시
RunDetails(run).show()
run.wait_for_completion()

실험을 처음 실행할 때는 Python 환경을 설정하는 데 시간이 다소 걸릴 수 있습니다. 후속 실행은 더 빠르게 진행됩니다.

실험이 완료되면 위젯에서 **azureml-logs/70_driver_log.txt** 출력 로그와 실행에서 생성된 메트릭을 확인합니다.

모든 실험에서와 마찬가지로 [Azure ML Studio](https://ml.azure.com)에서 실험 실행의 세부 정보를 확인할 수 있으며, 생성된 파일과 메트릭을 검색하는 코드를 작성할 수도 있습니다.

In [ ]:
# 로깅된 메트릭 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

여기서 학습시킨 모델은 **outputs** 폴더에 **diabetes_model.pkl** 파일로 저장되므로 등록할 수 있습니다.

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'SKLearn Estimator (tabular dataset)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## 파일 데이터 세트에서 모델 학습 진행

앞에서는 *테이블 형식* 데이터 세트의 학습 데이터를 사용해 모델을 학습시키는 방법을 살펴보았습니다. 이번에는 *파일* 데이터 세트를 사용하는 방법을 알아보겠습니다.

파일 데이터 세트를 사용할 때 스크립트에 전달되는 데이터 세트 입력은 파일 경로가 포함된 탑재 지점을 나타냅니다. 이러한 파일에서 데이터를 읽는 방법은 파일의 데이터 종류와 해당 데이터를 사용하여 수행하려는 작업에 따라 다릅니다. 당뇨병 CSV 파일의 경우에는 Python **glob** 모듈을 사용하여 데이터 세트에 의해 정의된 가상 탑재 지점에서 파일 목록을 만들 수 있으며, 모든 파일을 단일 데이터 프레임으로 연결된 Pandas 데이터 프레임으로 읽어들일 수 있습니다.

아래의 두 코드 셀을 실행하여 다음 항목을 만듭니다.

1. **diabetes_training_from_file_dataset** 폴더
2. 분류 모델을 학습시키는 스크립트. 이 스크립트는 전달된 파일 데이터 세트를 *입력*으로 사용합니다.

In [ ]:
import os

# 실험 파일용 폴더 만들기
experiment_folder = 'diabetes_training_from_file_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
import argparse
from azureml.core import Workspace, Dataset, Experiment, Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import glob

# 정규화 하이퍼 매개 변수 설정(스크립트에 인수로 전달됨)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 세트 로드
print("Loading Data...")
data_path = run.input_datasets['diabetes'] # 추정기 입력에서 학습 데이터 가져오기
all_files = glob.glob(data_path + "/*.csv")
diabetes = pd.concat((pd.read_csv(f) for f in all_files))

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 로지스틱 회귀 모델 학습
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 출력 폴더에 저장된 메모 파일이 실험 레코드에 자동으로 업로드됨
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

다음으로는 추정기에 데이터 세트를 전달하는 방식을 변경해야 합니다. 즉, 스크립트가 파일을 읽을 수 있는 탑재 지점을 정의해야 합니다. 데이터의 양이 많을 때는 보통 **as_mount** 메서드를 사용하여 데이터 세트 원본에서 파일을 직접 스트리밍합니다. 그러나 이 예제에서와 같이 로컬 컴퓨팅에서 실행할 때는 **as_download** 옵션을 사용하여 로컬 폴더에 데이터 세트 파일을 다운로드해야 합니다.

또한 **Dataset** 클래스는 **azureml-dataprep** 패키지에 정의되어 있으므로 실험 환경에 이 패키지를 포함해야 합니다.

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment
from azureml.widgets import RunDetails

# 스크립트 매개 변수 설정
script_params = {
    '--regularization': 0.1
}

# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes file dataset")

# 추정기 만들기
estimator = SKLearn(source_directory=experiment_folder,
                    entry_script='diabetes_training.py',
                    script_params=script_params,
                    compute_target = 'local',
                    inputs=[diabetes_ds.as_named_input('diabetes').as_download(path_on_compute='diabetes_data')], # Dataset 개체를 입력으로 전달
                    pip_packages=['azureml-dataprep[pandas]'] # dataprep 패키지 필요
                   )

# 실험 만들기
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# 실험 실행
run = experiment.submit(config=estimator)
# 실행 중에 실행 세부 정보 표시
RunDetails(run).show()
run.wait_for_completion()

실험이 완료되면 위젯에서 **azureml-logs/70_driver_log.txt** 출력 로그를 표시하여 파일 데이터 세트가 처리되었으며 데이터 파일이 다운로드되었는지 확인합니다.

모든 실험에서와 마찬가지로 [Azure ML Studio](https://ml.azure.com)에서 실험 실행의 세부 정보를 확인할 수 있으며, 생성된 파일과 메트릭을 검색하는 코드를 작성할 수도 있습니다.

In [ ]:
# 로깅된 메트릭 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

이번에도 학습시킨 모델을 등록해 보겠습니다.

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'SKLearn Estimator (file dataset)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **추가 정보**: 데이터 세트를 사용하는 학습에 대한 자세한 내용은 Azure ML 설명서의 [데이터 세트로 학습](https://docs.microsoft.com/azure/machine-learning/how-to-train-with-datasets)을 참조하세요.